In [63]:
import pandas as pd

In [64]:
# Membaca data dari file Excel
file_path = 'dataset sekunder/dataset_sekunder.xlsx'  # ganti dengan path file Excel kamu
sheet_names = [f'day {i}' for i in range(1, 21)]  # nama-nama sheet dari day 1 hingga day 20

# Membaca semua sheet menjadi dictionary of DataFrame
data = pd.read_excel(file_path, sheet_name=sheet_names)

# Menampilkan beberapa data dari sheet pertama untuk pengecekan
data['day 2'].head()

,DY,HR,suhu_udara,kelembapan_spesifik,kelembapan_relatif,curah_hujan,kecepatan_angin
0,23,8,25.65,15.50,71.25,0.01,3.42
1,23,9,28.15,14.53,57.56,0.00,3.33
2,23,10,30.23,13.73,48.44,0.00,3.13
3,23,11,31.89,13.37,42.81,0.00,3.03
4,23,12,32.94,13.37,40.38,0.00,2.94


In [65]:
# Gabungkan semua sheet menjadi satu DataFrame
combined_data = pd.concat([data[day].assign(day=day) for day in sheet_names])

# Cek data gabungan
combined_data.head()

,DY,HR,suhu_udara,kelembapan_spesifik,kelembapan_relatif,curah_hujan,kecepatan_angin,day
0,22,8,24.94,15.75,75.56,0.01,3.22,day 1
1,22,9,27.19,14.95,62.75,0.00,3.14,day 1
2,22,10,29.21,14.22,53.19,0.00,3.04,day 1
3,22,11,30.81,13.73,46.75,0.00,3.01,day 1
4,22,12,31.94,13.49,43.00,0.00,3.02,day 1


In [66]:
# Misalkan ada kolom 'time' yang berisi jam dalam format integer (8, 9, 10, dst.)
combined_data['time'] = pd.to_datetime(combined_data['HR'].astype(str) + ':00', format='%H:%M')

# Set 'time' sebagai index
combined_data.set_index('time', inplace=True)

# Cek data setelah setting index waktu
combined_data.head()

,DY,HR,suhu_udara,kelembapan_spesifik,kelembapan_relatif,curah_hujan,kecepatan_angin,day
time,,,,,,,,
1900-01-01 08:00:00,22,8,24.94,15.75,75.56,0.01,3.22,day 1
1900-01-01 09:00:00,22,9,27.19,14.95,62.75,0.00,3.14,day 1
1900-01-01 10:00:00,22,10,29.21,14.22,53.19,0.00,3.04,day 1
1900-01-01 11:00:00,22,11,30.81,13.73,46.75,0.00,3.01,day 1
1900-01-01 12:00:00,22,12,31.94,13.49,43.00,0.00,3.02,day 1


In [67]:
combined_data.drop(['DY', 'HR'], axis=1, inplace=True)

In [68]:
combined_data.describe()

,suhu_udara,kelembapan_spesifik,kelembapan_relatif,curah_hujan,kecepatan_angin
count,160.000000,160.000000,160.000000,160.000000,160.000000
mean,30.576125,14.702375,51.862063,0.104500,2.731063
std,2.597848,1.373641,12.156295,0.190711,0.852421
min,24.940000,11.470000,31.880000,0.000000,0.440000
25%,28.747500,13.490000,41.235000,0.000000,2.150000
50%,30.795000,14.890000,50.875000,0.030000,2.780000
75%,32.832500,15.620000,59.200000,0.140000,3.300000
max,35.190000,18.250000,84.500000,1.350000,4.750000
